In [1]:
import xarray as xr
import pathlib
import sys
import datetime
from data_config import (
    get_scratch_dir,
    get_dask_log_dir,
    get_dask_local_dir,
)

from process_files import (
    memory,
    get_case_metadata,
)
import numpy as np
import zarr

import pandas as pd
import tqdm
import dask.array as da

In [2]:
scratch = get_scratch_dir()
dask_log_directory = get_dask_log_dir()
dask_local_directory = get_dask_local_dir()

In [3]:
parent_dir = pathlib.Path.cwd().parent
sys.path.append(str(parent_dir))

In [4]:
import atlas

In [5]:
@memory.cache
def get_done_cases_df(today=datetime.datetime.today().date()):
    calc = atlas.global_irf_map(cdr_forcing="DOR", vintage="001")

    data = calc.df_case_status
    done = data.loc[data.archive]

    done_cases = done.index.to_list()
    done_cases.remove("smyle.cdr-atlas-v0.control.001")
    done_cases = sorted(done_cases)

    df = calc.df.loc[done_cases]
    return df, done_cases

In [6]:
%%time

df, done_cases = get_done_cases_df()
df

CPU times: user 1.12 ms, sys: 4.69 ms, total: 5.81 ms
Wall time: 6.1 ms


,blueprint,polygon,polygon_master,basin,start_date,cdr_forcing,cdr_forcing_file,simulation_key,refdate,stop_n,wallclock,curtail_output
case,,,,,,,,,,,,
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-01-01_00000.001,smyle,0.0,0.0,North_Atlantic_basin,1999-01,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_North_Atlantic_basin_000_1999-01-01_00000,0347-01-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-04-01_00001.001,smyle,0.0,0.0,North_Atlantic_basin,1999-04,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_North_Atlantic_basin_000_1999-04-01_00001,0347-04-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-07-01_00002.001,smyle,0.0,0.0,North_Atlantic_basin,1999-07,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_North_Atlantic_basin_000_1999-07-01_00002,0347-07-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-10-01_00003.001,smyle,0.0,0.0,North_Atlantic_basin,1999-10,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_North_Atlantic_basin_000_1999-10-01_00003,0347-10-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_001_1999-01-01_00004.001,smyle,1.0,1.0,North_Atlantic_basin,1999-01,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_North_Atlantic_basin_001_1999-01-01_00004,0347-01-01,15,10:00:00,True
...,...,...,...,...,...,...,...,...,...,...,...,...
smyle.cdr-atlas-v0.glb-dor_South_037_1999-01-01_01548.001,smyle,37.0,387.0,South,1999-01,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_South_037_1999-01-01_01548,0347-01-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_South_037_1999-04-01_01549.001,smyle,37.0,387.0,South,1999-04,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_South_037_1999-04-01_01549,0347-04-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_South_037_1999-10-01_01551.001,smyle,37.0,387.0,South,1999-10,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_South_037_1999-10-01_01551,0347-10-01,15,10:00:00,True


In [7]:
base_directory = pathlib.Path(
    "/global/cfs/projectdirs/m4746/Projects/Ocean-CDR-Atlas-v0/data/analysis"
)
base_directory

PosixPath('/global/cfs/projectdirs/m4746/Projects/Ocean-CDR-Atlas-v0/data/analysis')

In [8]:
def add_additional_coords(ds: xr.Dataset, case: str, case_metadata: pd.Series):
    polygon_master = int(case_metadata.polygon_master)
    if polygon_master < 0 or polygon_master > 689:
        raise ValueError(
            f"Polygon id must be in range [0, 690). Found polygon_id={polygon_master}"
        )

    # add as an integer coordinate
    polygon_id_coord = xr.DataArray(
        name="polygon_id",
        dims="polygon_id",
        data=[polygon_master],
        attrs={"long_name": "polygon ID"},
    ).astype("int32")

    # injenction date
    injection_date_coord = xr.DataArray(
        data=[int(case_metadata.start_date.split("-")[-1])],
        dims=["injection_date"],
        attrs={"long_name": "injection date", "units": "month of 1999"},
    ).astype("int32")

    renamed = ds.drop_vars("time").rename_dims(time="elapsed_time")

    return renamed.assign_coords(
        polygon_id=polygon_id_coord,
        injection_date=injection_date_coord,
    )


def expand_ensemble_dims(ds: xr.Dataset) -> xr.Dataset:
    """Add new dimensions across the ensemble"""

    copied = ds.copy()

    # all data variables should be ensemble variables
    for name in list(ds.data_vars):
        copied[name] = copied[name].expand_dims(["polygon_id", "injection_date"])

    # absolute time is a function of injection_date because of the different starting times
    # copied["time"] = copied["time"].expand_dims(["injection_date"])
    # copied["time_bound"] = copied["time_bound"].expand_dims(["injection_date"])

    return copied


def compute_dor_efficiency(ds: xr.Dataset) -> xr.Dataset:
    ds["DOR_efficiency"] = -ds.DIC_ADD_TOTAL / ds.DIC_FLUX
    return ds


def set_compression_encoding(ds: xr.Dataset) -> xr.Dataset:
    compressor = zarr.Zlib(level=1)

    for name, var in ds.variables.items():
        # avoid using NaN as a fill value, and avoid overflow errors in encoding
        if np.issubdtype(var.dtype, np.integer):
            ds[name].encoding = {"compressor": compressor, "_FillValue": 2_147_483_647}
        elif var.dtype == np.dtype("float32"):
            ds[name].encoding = {
                "compressor": compressor,
                "_FillValue": 9.969209968386869e36,
            }
        else:
            ds[name].encoding = {"compressor": compressor}

    return ds


def set_elapsed_time(ds: xr.Dataset):
    elapsed_time_integer_months = xr.DataArray(
        np.arange(180), dims=["elapsed_time"], attrs={"units": "months"}
    )
    ds["elapsed_time"] = elapsed_time_integer_months.astype("int32")
    return ds


def process_case(case: str, df: pd.DataFrame) -> xr.Dataset:
    case_metadata = get_case_metadata(case, df=df)
    path = base_directory / f"{case}.analysis.zarr"
    if not path.exists():
        return None
    ds = (
        xr.open_zarr(path, decode_timedelta=True)
        .pipe(add_additional_coords, case, case_metadata)
        .pipe(expand_ensemble_dims)
        .pipe(compute_dor_efficiency)
        .pipe(set_elapsed_time)
    )
    return ds[["DOR_efficiency", "elapsed_time"]].drop_vars(["time_delta"])

In [9]:
def create_empty_target_store():
    store1b_chunks_encoding_per_variable = {
        "DOR_efficiency": {
            "chunks": {"polygon_id": 1, "injection_date": 1, "elapsed_time": 180}
        },  # polygon_id: 1 injection_date: 1 elapsed_time: 180
        "polygon_id": {"chunks": {"polygon_id": 690}},  # polygon_id: 1
        "injection_date": {"chunks": {"injection_date": 1}},  # injection_date: 1
        "elapsed_time": {"chunks": {"elapsed_time": 180}},  # elapsed_time: 180
    }
    sizes_all_dims = {
        "elapsed_time": 180,
        "polygon_id": 690,
        "injection_date": 4,
    }

    placeholder = xr.Dataset()
    placeholder["elapsed_time"] = xr.DataArray(
        np.arange(180).astype("int32"), dims=["elapsed_time"], attrs={"units": "months"}
    )
    placeholder["polygon_id"] = xr.DataArray(
        np.arange(690).astype("int32"),
        dims=["polygon_id"],
        attrs={"long_name": "Polygon ID"},
    )
    placeholder["injection_date"] = xr.DataArray(
        np.array([1, 4, 7, 10], dtype="int32"),
        dims=["injection_date"],
        attrs={"long_name": "injection date", "units": "month of 1999"},
    )

    var_chunks = store1b_chunks_encoding_per_variable["DOR_efficiency"]["chunks"]
    var_dims = list(var_chunks.keys())
    var_sizes = {d: s for d, s in sizes_all_dims.items() if d in var_dims}
    var_shape = tuple(var_sizes.values())
    ordered_var_dims = list(var_sizes.keys())

    placeholder["DOR_efficiency"] = xr.DataArray(
        da.empty(
            shape=var_shape,
            chunks=var_chunks,
            dtype="float64",
        ),
        dims=ordered_var_dims,
    )
    placeholder = (
        placeholder.pipe(set_compression_encoding)
        .chunk(polygon_id=1, injection_date=1, elapsed_time=-1)
        .transpose("elapsed_time", "polygon_id", "injection_date")
    )

    return placeholder

In [10]:
store_path = "s3://carbonplan-share/dor-efficiency/store1b.zarr"

placeholder = create_empty_target_store()
placeholder

<xarray.Dataset> Size: 4MB
Dimensions:         (elapsed_time: 180, polygon_id: 690, injection_date: 4)
Coordinates:
  * elapsed_time    (elapsed_time) int32 720B 0 1 2 3 4 ... 175 176 177 178 179
  * polygon_id      (polygon_id) int32 3kB 0 1 2 3 4 5 ... 685 686 687 688 689
  * injection_date  (injection_date) int32 16B 1 4 7 10
Data variables:
    DOR_efficiency  (elapsed_time, polygon_id, injection_date) float64 4MB dask.array<chunksize=(180, 1, 1), meta=np.ndarray>

In [11]:
placeholder.to_zarr(
    store_path, consolidated=True, zarr_format=2, mode="w", compute=False
)

Delayed('_finalize_store-ba9ef5ac-074e-4ef7-ba8e-0d35d728a721')

In [ ]:
%%time

cases_without_data = []
for case in tqdm.tqdm(done_cases):
    single_ds = process_case(case=case, df=df)
    if single_ds:
        single_ds.to_zarr(store_path, region="auto")
    else:
        cases_without_data.append(case)

# ds = xr.combine_by_coords(dsets, combine_attrs='drop_conflicts').chunk(polygon_id=-1, injection_date=1, elapsed_time=-1).transpose('elapsed_time', 'polygon_id', 'injection_date')
# ds

 10%|█         | 148/1414 [00:46<06:42,  3.14it/s]

In [ ]:
ds = xr.open_dataset(store_path, engine="zarr", chunks={})
ds.sel(polygon_id=0).DOR_efficiency.plot(col_wrap=4, col="injection_date")

In [ ]:
# dset = xr.open_dataset("https://carbonplan-oae-efficiency.s3.us-west-2.amazonaws.com/store1b.zarr/", engine='zarr', chunks={})
# dset